<a href="https://colab.research.google.com/github/HosseinPakrooh/deep-learning--Recognizing-duplicate-questions/blob/main/ESEM_seprate22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/drive')
!ls/drive/my\drive

Mounted at /drive
/bin/bash: ls/drive/mydrive: No such file or directory


In [ ]:
import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras
import numpy as np
import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

In [ ]:
Train_QUESTION_PAIRS_FILE = '/drive/My Drive/Colab Notebooks/Pakrooh/medium_link_prediction_noClue_shuffled_train.csv'
Test_QUESTION_PAIRS_FILE =  '/drive/My Drive/Colab Notebooks/Pakrooh/medium_link_prediction_noClue_shuffled_test.csv'
GLOVE_FILE = 'glove.840B.300d.txt'
MAX_NB_WORDS = 146090
MAX_SEQUENCE_LENGTH_body = 70
EMBEDDING_DIM = 300

In [ ]:
pip install python-Levenshtein==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 5.4 MB/s 
Reason for being yanked: Insecure, upgrade to 0.12.1
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp37-cp37m-linux_x86_64.whl size=145925 sha256=068a5eb98bd0d787a962676dfa084ce279be64d59a600f30ad0961d1626ef601
  Stored in directory: /root/.cache/pip/wheels/f0/9b/13/49c281164c37be18343230d3cd0fca29efb23a493351db0009
Successfully built python-Levenshtein


In [ ]:
import re
import string
# import Levenshtein
import nltk
nltk.download('stopwords')

def convertCamelCase(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    finals = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    #     print(finals)
    return finals


def text_to_wordlist(text, remove_stopwords=True, stem_words=False, remove_punc=True):
    # URL remove
    #     print(type(text))
    text = str(text)
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'URL', text)
    # split words:
    text = text.split()
    text2 = []
    for t in text:
        splited = re.split("[.,' \-!?:#^%*[$\]+|_`\)=<;{\"&>@/~(\\}\\\]+", t)
        text2.extend(splited)
    not_digits=[]
    for i in text2:
        if i.isdigit():
            not_digits.append("cc")
        else:
            not_digits.append(i)
    text4 = []
    for c in not_digits:
        if len(c) > 1:
            if c[1].islower():
                for k in c[1:]:
                    if not k.islower():
                        text4.append(convertCamelCase(c))
                        break;
                else:
                    text4.append(c)
            else:
                text4.append(c)
        else:
            text4.append(c)

    text5 = []
    for t in text4:
        splited = re.split("_", t)
        text5.extend(splited)
    text6=[]
    for x in text5:
        if len(x)>1:
            text6.append(x)

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text6 if not w in stops]
    # print("after stop ", text)
    # print("after stop word", text)
    text = " ".join(text)

    text = re.sub("  ", " ", text)
    text = re.sub("   ", " ", text)
    text = re.sub("    ", " ", text)
    text = re.sub("[0-9]+", "CC", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"couldn't", "could not ", text)
    text = re.sub(r"doesn’t", "does not ", text)
    text = re.sub(r"don't", "do not ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    text = re.sub(r"“", "", text)
    text = re.sub(r"”", "", text)

    if remove_punc:
        exclude = set(string.punctuation)
        text = ''.join(ch for ch in text if ch not in exclude)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)

    text = text.lower()

    return (text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
test = text_to_wordlist("a5HRsatX is \"qood\" or 'WOOd or EooOd or rooOood AaokkE ยง15 !?,,'[]{} to a the - _", remove_stopwords=True, remove_punc=True)
print(test)

acchrsatx qood wood eoo od roo oood aaokk ยงcc


In [ ]:
preprocessed_data = "/drive/My Drive/Colab Notebooks/Pakrooh/"

# Reading and Preprocessing the Trainset

In [ ]:
train_q1_Id =[]
train_q1_Title =[]
train_q1_Body =[]
train_q1_AcceptedAnswerId =[]
train_q1_AcceptedAnswerBody=[]
train_q1_AnswersIdList=[]
train_q1_AnswersBody=[]
train_q1_Tags=[]
train_q2_Id=[]
train_q2_Title=[]
train_q2_Body=[]
train_q2_AcceptedAnswerId=[]
train_q2_AcceptedAnswerBody=[]
train_q2_AnswersIdList=[]
train_q2_AnswersBody=[]
train_q2_Tags=[]
train_class=[]

import pandas as pd
df = pd.read_csv( Train_QUESTION_PAIRS_FILE, sep=",", lineterminator='\n' )
df.columns=['id', 'q1_Id', 'q1_Title', 'q1_Body', 'q1_BodyCode','q1_AcceptedAnswerId', 'q1_AcceptedAnswerBody', 'q1_AcceptedAnswerCode', 'q1_AnswersIdList', 'q1_AnswersBody', 'q1_AnswersCode', 'q1_Tags', 'q2_Id', 'q2_Title', 'q2_Body', 'q2_BodyCode', 'q2_AcceptedAnswerId', 'q2_AcceptedAnswerBody', 'q2_AcceptedAnswerCode', 'q2_AnswersIdList', 'q2_AnswersBody', 'q2_AnswersCode', 'q2_Tags', 'class' ]
count =0
for index, row in df.iterrows():
    q1_Title=text_to_wordlist(str(row['q1_Title']))+" "+text_to_wordlist(str(row[ 'q1_Tags']))
    # q1_Tags=text_to_wordlist(str(row[ 'q1_Tags']))
    q1_Body = text_to_wordlist(str(row['q1_Body']), remove_stopwords=True, stem_words=False)
    q1_Answer=text_to_wordlist(str(row['q1_AcceptedAnswerBody'])+" "+str(row[ 'q1_AnswersBody'])\
                               +" "+str(row[ 'q1_AcceptedAnswerCode']))
    train_q1_Title.append(q1_Title)
    # train_q1_Tags.append(q1_Tags)
    train_q1_Body.append(q1_Body)
    train_q1_AnswersBody.append(q1_Answer)

    q2_Title=text_to_wordlist(str(row['q2_Title']))+" "+text_to_wordlist(str(row[ 'q2_Tags']))
    # q2_Tags=text_to_wordlist(str(row[ 'q2_Tags']))
    q2_Body = text_to_wordlist(str(row['q2_Body']), remove_stopwords=True, stem_words=False)
    q2_Answer=text_to_wordlist(str(row['q2_AcceptedAnswerBody'])+" "+str(row[ 'q2_AnswersBody'])\
                               +" "+str(row[ 'q2_AcceptedAnswerCode']))
    train_q2_Title.append(q2_Title)
    # train_q2_Tags.append(q2_Tags)
    train_q2_Body.append(q2_Body)
    train_q2_AnswersBody.append(q2_Answer)
    train_class.append(row['class'])

    count+=1
print(count)

32000


In [ ]:
train_q2_Body[0]

'trying parse date string modification date file ftp server following code get log can anyone please help'

In [ ]:
preprocessed_data = "/drive/My Drive/Colab Notebooks/Pakrooh/"

In [ ]:
import pickle as pk

with open(preprocessed_data+"train_q1_Body.pkl", 'wb') as fb:
    pk.dump(train_q1_Body, fb)

with open(preprocessed_data+"train_q2_Body.pkl", 'wb') as fb:
    pk.dump(train_q2_Body, fb)

with open(preprocessed_data+"train_class.pkl", 'wb') as fb:
    pk.dump(train_class, fb)

print("train_class ", train_class[:2])

train_class  ['duplicate\n', 'duplicate\n']


In [ ]:
test_q1_Id =[]
test_q1_Title =[]
test_q1_Body =[]
test_q1_AcceptedAnswerId =[]
test_q1_AcceptedAnswerBody=[]
test_q1_AnswersIdList=[]
test_q1_AnswersBody=[]
test_q1_Tags=[]
test_q2_Id=[]
test_q2_Title=[]
test_q2_Body=[]
test_q2_AcceptedAnswerId=[]
test_q2_AcceptedAnswerBody=[]
test_q2_AnswersIdList=[]
test_q2_AnswersBody=[]
test_q2_Tags=[]
test_class=[]

import pandas as pd
df = pd.read_csv(Test_QUESTION_PAIRS_FILE, sep=",", lineterminator='\n' )
df.columns=['id', 'q1_Id', 'q1_Title', 'q1_Body', 'q1_BodyCode','q1_AcceptedAnswerId', 'q1_AcceptedAnswerBody', 'q1_AcceptedAnswerCode', 'q1_AnswersIdList', 'q1_AnswersBody', 'q1_AnswersCode', 'q1_Tags', 'q2_Id', 'q2_Title', 'q2_Body', 'q2_BodyCode', 'q2_AcceptedAnswerId', 'q2_AcceptedAnswerBody', 'q2_AcceptedAnswerCode', 'q2_AnswersIdList', 'q2_AnswersBody', 'q2_AnswersCode', 'q2_Tags', 'class' ]
count =0
for index, row in df.iterrows():
    q1_Title=text_to_wordlist(str(row['q1_Title']))+" " +text_to_wordlist(str(row[ 'q1_Tags']))
    # q1_Tags=
    q1_Body = text_to_wordlist(str(row['q1_Body']), remove_stopwords=True, stem_words=False)
    q1_Answer=text_to_wordlist(str(row['q1_AcceptedAnswerBody'])+" "+str(row[ 'q1_AnswersBody'])\
                               +" "+str(row[ 'q1_AcceptedAnswerCode']))
    test_q1_Title.append(q1_Title)
    # test_q1_Tags.append(q1_Tags)
    test_q1_Body.append(q1_Body)
    test_q1_AnswersBody.append(q1_Answer)

    q2_Title=text_to_wordlist(str(row['q2_Title']))+ " "+text_to_wordlist(str(row[ 'q2_Tags']))
    # q2_Tags=
    q2_Body = text_to_wordlist(str(row['q2_Body']), remove_stopwords=True, stem_words=False)
    q2_Answer=text_to_wordlist(str(row['q2_AcceptedAnswerBody'])+" "+str(row[ 'q2_AnswersBody'])\
                               +" "+str(row[ 'q2_AcceptedAnswerCode']))
    test_q2_Title.append(q2_Title)
    # test_q2_Tags.append(q2_Tags)

    test_q2_Body.append(q2_Body)
    test_q2_AnswersBody.append(q2_Answer)
    test_class.append(row['class'])

    count+=1
print(count)

8000


In [ ]:
with open(preprocessed_data+"test_q1_Body.pkl", 'wb') as fb:
    pk.dump(test_q1_Body, fb)


with open(preprocessed_data+"test_q2_Body.pkl", 'wb') as fb:
    pk.dump(test_q2_Body, fb)

with open(preprocessed_data+"test_class.pkl", 'wb') as fb:
    pk.dump(test_class, fb)

print("test_class ", test_class[:2])

test_class  ['duplicate\n', 'isolated\n']


In [ ]:
import pickle as pk

In [ ]:
#@title loading Preprocessed Train data

preprocessed_data = "/drive/My Drive/Colab Notebooks/Pakrooh/"

train_q1_Body= pk.load(open(preprocessed_data+"train_q1_Body.pkl", 'rb'))

train_q2_Body= pk.load(open(preprocessed_data+"train_q2_Body.pkl", 'rb'))

train_labels= pk.load(open(preprocessed_data+"train_class.pkl", 'rb'))

In [ ]:
# train_classes= pk.load(open(preprocessed_data+"train_class.pkl", 'rb'))


In [ ]:
train_q2_Body[0]

'trying parse date string modification date file ftp server following code get log can anyone please help'

In [ ]:
#@title encoding labels of Train Set

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_class)
encoded_Y = encoder.transform(train_class)
# convert integers to dummy variables (i.e. one hot encoded)
train_Y = np_utils.to_categorical(encoded_Y)
trainY=encoded_Y
train_Y


array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
#@title loading preprocessed Test Data
with open(preprocessed_data+"test_q1_Body.pkl", 'rb') as fb:
    test_q1_Body= pk.load(fb)

with open(preprocessed_data+"test_q2_Body.pkl", 'rb') as fb:
    test_q2_Body= pk.load(fb)

with open(preprocessed_data+"test_class.pkl", 'rb') as fb:
    test_labels= pk.load(fb)

In [ ]:
with open(preprocessed_data+"test_class.pkl", 'rb') as fb:
    test_labels= pk.load(fb)

In [ ]:
#@title encoding labels of Test Set

encoded_Y = encoder.transform(test_class)
# convert integers to dummy variables (i.e. one hot encoded)
test_Y = np_utils.to_categorical(encoded_Y)
testY=encoded_Y
test_Y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [ ]:
len(test_Y)

8000

In [ ]:
encoded_Y

array([1, 3, 3, ..., 3, 0, 3])

In [ ]:
train_Y

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
# label_tokenizer = Tokenizer()
# myset = set(test_labels)
# print(myset)
# label_tokenizer.fit_on_texts(list(train_labels))
# training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
# test_label_seq = np.array(label_tokenizer.texts_to_sequences(test_labels))
# print(min(training_label_seq))
# print(training_label_seq.shape)

In [ ]:
train_labels[0]

'duplicate\n'

In [ ]:
test_labels[0]

'duplicate\n'

#TOKEINZING WHOLE QUESTIONS

In [ ]:
questions =  test_q1_Body +  \
test_q2_Body +  \
 train_q1_Body + \
train_q2_Body + train_q1_AnswersBody+train_q2_AnswersBody+test_q1_AnswersBody +test_q2_AnswersBody\
+train_q1_Title+train_q2_Title+test_q1_Title+test_q2_Title

MAX_NB_WORDS=20000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) #Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)

train_q1_Body_word_sequences = tokenizer.texts_to_sequences(train_q1_Body)
train_q2_Body_word_sequences = tokenizer.texts_to_sequences(train_q2_Body)
test_q1_Body_word_sequences = tokenizer.texts_to_sequences(test_q1_Body)
test_q2_Body_word_sequences = tokenizer.texts_to_sequences(test_q2_Body)

##############################################################
# train_q1_Tags_word_sequences = tokenizer.texts_to_sequences(train_q1_Tags)
# train_q2_Tags_word_sequences = tokenizer.texts_to_sequences(train_q2_Tags)
# test_q1_Tags_word_sequences = tokenizer.texts_to_sequences(test_q1_Tags)
# test_q2_Tags_word_sequences = tokenizer.texts_to_sequences(test_q2_Tags)

####################################################################
train_q1_Title_word_sequences = tokenizer.texts_to_sequences(train_q1_Title)
train_q2_Title_word_sequences = tokenizer.texts_to_sequences(train_q2_Title)
test_q1_Title_word_sequences = tokenizer.texts_to_sequences(test_q1_Title)
test_q2_Title_word_sequences = tokenizer.texts_to_sequences(test_q2_Title)
########################################################################
train_q1_AnswersBody_word_sequences = tokenizer.texts_to_sequences(train_q1_AnswersBody)
train_q2_AnswersBody_word_sequences = tokenizer.texts_to_sequences(train_q2_AnswersBody)
test_q1_AnswersBody_word_sequences = tokenizer.texts_to_sequences(test_q1_AnswersBody)
test_q2_AnswersBody_word_sequences = tokenizer.texts_to_sequences(test_q2_AnswersBody)


In [ ]:
test_q2_Body_word_sequences[1000]

In [ ]:
len(train_q2_Body_word_sequences[0])

17

In [ ]:

word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))
print(test_q2_Body_word_sequences[6000])
print(len(train_q1_Body_word_sequences))
# trainX=[]
# testX=[]
# # trainX.append(test_q1_Body_word_sequences[0]+test_q2_Body_word_sequences[0])
# for i in range(len(train_q1_Body_word_sequences)):
#   trainX.append(train_q1_Body_word_sequences[i]+train_q2_Body_word_sequences[i])
# for i in range(len(test_q1_Body_word_sequences)):
#   testX.append(test_q1_Body_word_sequences[i]+test_q2_Body_word_sequences[i])


Words in index: 92240
[17, 40, 418, 6759, 12, 327, 6155, 68, 6410, 114, 6, 1150, 984, 2303, 2145, 2940, 18, 2937, 154, 997, 64, 157, 105, 3694, 3920, 671, 1303, 183, 106, 505, 184, 77, 31, 473, 293, 6, 85, 250, 787, 2937, 64, 10177, 926, 2303, 207, 105, 2937, 207, 183, 42]
32000


In [ ]:
######################
MAX_SEQUENCE_LENGTH_body=70
MAX_SEQUENCE_LENGTH_Title=20
MAX_SEQUENCE_LENGTH_AnswersBody=180
########################
print('Shape of train question2 data tensor:', len(train_q1_Body_word_sequences))
print('Shape of train question2 data tensor:', len(train_q2_Body_word_sequences))

print('Shape of train question2 data tensor:', len(test_q1_Body_word_sequences))

print('Shape of train question2 data tensor:', len(test_q2_Body_word_sequences))

q1_train_body = pad_sequences(train_q1_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)
q2_train_body = pad_sequences(train_q2_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)
q1_test_body = pad_sequences(test_q1_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)
q2_test_body = pad_sequences(test_q2_Body_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_body)

#####################
q1_train_Title = pad_sequences(train_q1_Title_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_Title)
q2_train_Title = pad_sequences(train_q2_Title_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_Title)
q1_test_Title = pad_sequences(test_q1_Title_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_Title)
q2_test_Title = pad_sequences(test_q2_Title_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_Title)
###################################
q1_train_AnswersBody = pad_sequences(train_q1_AnswersBody_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_AnswersBody)
q2_train_AnswersBody = pad_sequences(train_q2_AnswersBody_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_AnswersBody)
q1_test_AnswersBody = pad_sequences(test_q1_AnswersBody_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_AnswersBody)
q2_test_AnswersBody = pad_sequences(test_q2_AnswersBody_word_sequences, maxlen=MAX_SEQUENCE_LENGTH_AnswersBody)

print("After padding . .. . ")
print('Shape of train question2 data tensor:', q1_train_body.shape)

print('Shape of train question2 data tensor:', q2_train_body.shape)

print('Shape of train question2 data tensor:', q1_test_body.shape)

print('Shape of train question2 data tensor:', q2_test_body.shape)
#######################################################################


# trainX = pad_sequences(trainX, maxlen=150)

# testX = pad_sequences(testX, maxlen=150)
# trainX.shape

Shape of train question2 data tensor: 32000
Shape of train question2 data tensor: 32000
Shape of train question2 data tensor: 8000
Shape of train question2 data tensor: 8000
After padding . .. . 
Shape of train question2 data tensor: (32000, 70)
Shape of train question2 data tensor: (32000, 70)
Shape of train question2 data tensor: (8000, 70)
Shape of train question2 data tensor: (8000, 70)


In [ ]:
print(train_Y.shape)

(32000, 4)


In [ ]:
import keras

In [ ]:
word_index

In [ ]:
!wget https://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip -q glove.840B.300d.zip

--2022-10-22 17:07:47--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2022-10-22 17:07:47--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  5.01MB/s    in 6m 49s  

2022-10-22 17:14:38 (5.07 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [ ]:
EMBEDDING_DIM = 300
GLOVE_FILE='glove.840B.300d.txt'    #'glove.840B.300d.txt'
print("Processing", GLOVE_FILE)

embeddings_index = {}
with open(GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

Processing glove.840B.300d.txt


FileNotFoundError: ignored

In [ ]:
nb_words = min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    unknown_vector = embeddings_index.get("java")
    if embedding_vector is not  None:
        word_embedding_matrix[i] = embedding_vector
    else:
#         word_embedding_matrix[i] =unknown_vector
        print(word)

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

xecc
xccc
logccj
xacc
ioexception
xbcc
javase
slfccj
ccxcc
jdkcc
strutscc
jlabel
jsoup
basecc
jsonobject
testcc
fxml
ccrd
xccd
ccnd
sqlexception
mpcc
tomcatcc
ccth
externalizable
ccst
pdfbox
jcombobox
ccbit
jaxbcontext
logccjcc
argcc
ccmb
listcc
joptionpane
ccucc
jsonarray
jscrollpane
jooq
jtextarea
unmarshaller
asynctask
ccms
ljava
xccs
valuecc
pkcscc
resteasy
mdcc
classcc
uimanager
ipvcc
jvms
arrcc
pointcc
jrecc
ccgb
shacc
ognl
jccee
methodcc
junitcc
urlconnection
kotlin
ycc
jpql
cctcc
arraycc
graphicscc
jccse
wccc
jcomponent
fasterxml
xcct
jdialog
wildfly
strcc
indexcc
cccpcc
djava
axiscc
bdoughan
thymeleaf
usercc
logcc
editcc
rccd
openqa
mcce
ยงcc
opennlp
jfilechooser
pojos
jdbccc
technetwork
ccz
jasperreports
urlclassloader
htmlunit
xbb
zcc
javassist
hamcrest
xcca
utfcc
fieldcc
paramcc
textcc
apidocs
jaxbelement
jaxbcc
threadcc
ojdbccc
mybatis
numcc
neoccj
docxccj
gccd
oauthcc
itextpdf
systemcc
xcce
httpcc
stringcc
namecc
domccj
keycc
jcifs
jaxrs
jres
sscce
nashorn
jtextpane
image

In [ ]:
#@title stable version so far
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.metrics import classification_report
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional

# nb_words=len(word_index)+1

nb_words=MAX_NB_WORDS+1
embedding_dim=300
embedding_layer = Embedding(
    nb_words,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(word_embedding_matrix),
    trainable=False


)
neuron=100
shared_lstm = Bidirectional(LSTM(neuron, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
# create model
sequence_1_Title = Input(shape=(MAX_SEQUENCE_LENGTH_Title,), dtype='float32')
embedding_Title_1 = embedding_layer(sequence_1_Title)
lstm_layer_Title_1=shared_lstm(embedding_Title_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Title = Input(shape=(MAX_SEQUENCE_LENGTH_Title,), dtype='float32')
embedding_Title_2 = embedding_layer(sequence_2_Title)
lstm_layer_Title_2=shared_lstm(embedding_Title_2)
#############
sequence_1_Body = Input(shape=(MAX_SEQUENCE_LENGTH_body,), dtype='float32')
embedding_Body_1 = embedding_layer(sequence_1_Body)
lstm_layer_Body_1=shared_lstm(embedding_Body_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Body = Input(shape=(MAX_SEQUENCE_LENGTH_body,), dtype='float32')
embedding_Body_2 = embedding_layer(sequence_2_Body)
lstm_layer_Body_2=shared_lstm(embedding_Body_2)



############
sequence_1_Answer = Input(shape=(MAX_SEQUENCE_LENGTH_AnswersBody,), dtype='float32')
embedding_Answer_1 = embedding_layer(sequence_1_Answer)
lstm_layer_Answer_1=shared_lstm(embedding_Answer_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Answer = Input(shape=(MAX_SEQUENCE_LENGTH_AnswersBody,), dtype='float32')
embedding_Answer_2 = embedding_layer(sequence_2_Answer)
lstm_layer_Answer_2=shared_lstm(embedding_Answer_2)

#########


#########
# att1=keras.layers.MultiHeadAttention(num_heads=1,key_dim=1)(lstm_layer1,lstm_layer1,lstm_layer1)
# att2=keras.layers.MultiHeadAttention(num_heads=1,key_dim=1)(lstm_layer2,lstm_layer2,lstm_layer2)
flat1_t = keras.layers.Flatten()(lstm_layer_Title_1)
flat2_t = keras.layers.Flatten()(lstm_layer_Title_2)
flat1_b = keras.layers.Flatten()(lstm_layer_Body_1)
flat2_b = keras.layers.Flatten()(lstm_layer_Body_2)
flat1_a = keras.layers.Flatten()(lstm_layer_Answer_1)
flat2_a = keras.layers.Flatten()(lstm_layer_Answer_2)
t_cos=keras.layers.Dot(axes=1, normalize=True)([lstm_layer_Title_1,lstm_layer_Title_2])
print(t_cos.shape)
flat_title_cosin = keras.layers.Flatten()(t_cos)
b_cos=keras.layers.Dot(axes=1, normalize=True)([lstm_layer_Body_1,lstm_layer_Body_2])
flat_body_cosin = keras.layers.Flatten()(b_cos)
a_cos=keras.layers.Dot(axes=1, normalize=True)([lstm_layer_Answer_1,lstm_layer_Answer_2])
flat_answer_cosin = keras.layers.Flatten()(a_cos)
# print("X",train_data_1[0])
# merged = keras.layers.concatenate([lstm_layer1, lstm_layer2], axis=-1)

merged = keras.layers.concatenate([flat_title_cosin,flat_body_cosin,flat1_t, flat_answer_cosin,flat2_t,flat1_b, flat2_b,flat1_a, flat2_a])
# print("X",train_data_1[0])
# merged = keras.layers.concatenate([lstm_layer1, lstm_layer2], axis=-1)

# merged = keras.layers.concatenate([flat1_t, flat2_t,flat1_b, flat2_b,flat1_a, flat2_a])
merged=Dense(50,activation='relu')(merged)

# merged = keras.layers.Flatten()(merged)
merged=keras.layers.Dropout(0.2)(merged)
output=Dense(4, activation='softmax')(merged)
model = Model(inputs=[sequence_1_Title,sequence_2_Title,sequence_1_Body,sequence_2_Body,sequence_1_Answer,sequence_2_Answer],outputs=output)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for i in range(15):
  print("Epoch:",i)
  history=model.fit([q1_train_Title, q2_train_Title, q1_train_body,q2_train_body,q1_train_AnswersBody,q2_train_AnswersBody]\
                  ,train_Y, validation_data=([q1_test_Title, q2_test_Title, q1_test_body,q2_test_body,q1_test_AnswersBody,q2_test_AnswersBody],test_Y),\
                  epochs=1, batch_size=512)#class_weight={0:.4,1:.2,2:.2,3:.3}
  model.save_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')
# model.save_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')

(None, 200, 200)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 70)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 70)]         0           []                               
                                                                             

In [ ]:
from sklearn.metrics import classification_report
model.load_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')

preds = model.predict([q1_test_Title, q2_test_Title, q1_test_body,q2_test_body,q1_test_AnswersBody,q2_test_AnswersBody],batch_size=512)
preds_train = model.predict([q1_train_Title, q2_train_Title, q1_train_body,q2_train_body,q1_train_AnswersBody,q2_train_AnswersBody],batch_size=512)
print("For Test",classification_report(testY, np.argmax(preds, axis=1), labels=[0,1,2,3]))
print("For Train",classification_report(trainY, np.argmax(preds_train, axis=1), labels=[0,1,2,3])

63/63 [==============================] - 33s 520ms/step


In [ ]:
np.savetxt("a.csv",preds)

In [ ]:
print("For Test",classification_report(testY, np.argmax(preds, axis=1), labels=[0,1,2,3]))
print("For Train",classification_report(trainY, np.argmax(preds_train, axis=1), labels=[0,1,2,3]))

For Test               precision    recall  f1-score   support

           0       0.40      0.33      0.36      2000
           1       0.54      0.57      0.55      2000
           2       0.47      0.56      0.51      2000
           3       0.82      0.76      0.79      2000

    accuracy                           0.56      8000
   macro avg       0.56      0.56      0.56      8000
weighted avg       0.56      0.56      0.56      8000

For Train               precision    recall  f1-score   support

           0       0.96      0.98      0.97      8000
           1       0.98      0.97      0.98      8000
           2       0.98      0.97      0.97      8000
           3       0.99      0.99      0.99      8000

    accuracy                           0.98     32000
   macro avg       0.98      0.98      0.98     32000
weighted avg       0.98      0.98      0.98     32000



In [ ]:
print("new")

In [ ]:
import gc

In [ ]:
del model

In [ ]:
gc.collect()

6526

TOP CODES are stable version


In [ ]:

#@title Testing additional features such global max pooling
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.metrics import classification_report
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.utils.vis_utils import plot_model

# nb_words=len(word_index)+1

nb_words=MAX_NB_WORDS+1
embedding_dim=300
embedding_layer = Embedding(
    nb_words,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(word_embedding_matrix),
    trainable=False


)
neuron=100
shared_lstm = Bidirectional(LSTM(neuron, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
# create model
sequence_1_Title = Input(shape=(MAX_SEQUENCE_LENGTH_Title,), dtype='float32')
embedding_Title_1 = embedding_layer(sequence_1_Title)
lstm_layer_Title_1=shared_lstm(embedding_Title_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Title = Input(shape=(MAX_SEQUENCE_LENGTH_Title,), dtype='float32')
embedding_Title_2 = embedding_layer(sequence_2_Title)
lstm_layer_Title_2=shared_lstm(embedding_Title_2)
#############
sequence_1_Body = Input(shape=(MAX_SEQUENCE_LENGTH_body,), dtype='float32')
embedding_Body_1 = embedding_layer(sequence_1_Body)
lstm_layer_Body_1=shared_lstm(embedding_Body_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Body = Input(shape=(MAX_SEQUENCE_LENGTH_body,), dtype='float32')
embedding_Body_2 = embedding_layer(sequence_2_Body)
lstm_layer_Body_2=shared_lstm(embedding_Body_2)



############
sequence_1_Answer = Input(shape=(MAX_SEQUENCE_LENGTH_AnswersBody,), dtype='float32')
embedding_Answer_1 = embedding_layer(sequence_1_Answer)
lstm_layer_Answer_1=shared_lstm(embedding_Answer_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Answer = Input(shape=(MAX_SEQUENCE_LENGTH_AnswersBody,), dtype='float32')
embedding_Answer_2 = embedding_layer(sequence_2_Answer)
lstm_layer_Answer_2=shared_lstm(embedding_Answer_2)

#########


#########
# att1=keras.layers.MultiHeadAttention(num_heads=1,key_dim=1)(lstm_layer1,lstm_layer1,lstm_layer1)
# att2=keras.layers.MultiHeadAttention(num_heads=1,key_dim=1)(lstm_layer2,lstm_layer2,lstm_layer2)
# flat1_t = keras.layers.Flatten()(lstm_layer_Title_1)
# flat1_t=keras.layers.GlobalMaxPool1D((lstm_layer_Title_1))
# distributed = TimeDistributed(Dense(1))(lstm_layer_Title_1)
flat1_t = keras.layers.AveragePooling1D()(lstm_layer_Title_1)
flat1_t=keras.layers.Flatten()(flat1_t)
print("shape", flat1_t.shape)
flat2_t = keras.layers.AveragePooling1D()(lstm_layer_Title_2)
flat2_t=keras.layers.Flatten()(flat2_t)
print("shape", flat2_t.shape)


# flat2_t = keras.layers.Flatten()(lstm_layer_Title_2)
# print("shape2,",flat2_t.shape)
flat1_b = keras.layers.AveragePooling1D()(lstm_layer_Body_1)
flat1_b = keras.layers.Flatten()(flat1_b)
flat2_b = keras.layers.AveragePooling1D()(lstm_layer_Body_2)
flat2_b = keras.layers.Flatten()(flat2_b)

# flat2_b = keras.layers.Flatten()(lstm_layer_Body_2)

flat1_a = keras.layers.AveragePooling1D()(lstm_layer_Answer_1)
flat1_a = keras.layers.Flatten()(flat1_a)

flat2_a = keras.layers.AveragePooling1D()(lstm_layer_Answer_2)
flat2_a = keras.layers.Flatten()(flat2_a)
# flat1_a = keras.layers.Flatten()(lstm_layer_Answer_1)
# flat2_a = keras.layers.Flatten()(lstm_layer_Answer_2)
t_cos=keras.layers.Dot(axes=1, normalize=True)([flat1_t,flat2_t])
print(t_cos.shape)
flat_title_cosin = keras.layers.Flatten()(t_cos)
b_cos=keras.layers.Dot(axes=1, normalize=True)([flat1_b,flat1_b])
flat_body_cosin = keras.layers.Flatten()(b_cos)
a_cos=keras.layers.Dot(axes=1, normalize=True)([flat1_a,flat1_a])
flat_answer_cosin = keras.layers.Flatten()(a_cos)
# print("X",train_data_1[0])
# merged = keras.layers.concatenate([lstm_layer1, lstm_layer2], axis=-1)

merged = keras.layers.concatenate([flat_title_cosin,flat_body_cosin,flat1_t, flat_answer_cosin,flat2_t,flat1_b, flat2_b,flat1_a, flat2_a])
# print("X",train_data_1[0])
# merged = keras.layers.concatenate([lstm_layer1, lstm_layer2], axis=-1)

# merged = keras.layers.concatenate([flat1_t, flat2_t,flat1_b, flat2_b,flat1_a, flat2_a])
merged=Dense(50,activation='relu')(merged)

# merged = keras.layers.Flatten()(merged)
merged=keras.layers.Dropout(0.2)(merged)
output=Dense(4, activation='softmax')(merged)
model = Model(inputs=[sequence_1_Title,sequence_2_Title,sequence_1_Body,sequence_2_Body,sequence_1_Answer,sequence_2_Answer],outputs=output)
model.summary()

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for i in range(15):
  print("Epoch:",i)
  history=model.fit([q1_train_Title, q2_train_Title, q1_train_body,q2_train_body,q1_train_AnswersBody,q2_train_AnswersBody]\
                  ,train_Y, validation_data=([q1_test_Title, q2_test_Title, q1_test_body,q2_test_body,q1_test_AnswersBody,q2_test_AnswersBody],test_Y),\
                  epochs=1, batch_size=512)#class_weight={0:.4,1:.2,2:.2,3:.3}
  model.save_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')
# model.save_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')

In [ ]:
from sklearn.metrics import classification_report
model.load_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')

preds = model.predict([q1_test_Title, q2_test_Title, q1_test_body,q2_test_body,q1_test_AnswersBody,q2_test_AnswersBody],batch_size=512)
preds_train = model.predict([q1_train_Title, q2_train_Title, q1_train_body,q2_train_body,q1_train_AnswersBody,q2_train_AnswersBody],batch_size=512)
print("For Test",classification_report(testY, np.argmax(preds, axis=1), labels=[0,1,2,3]))
print("For Train",classification_report(trainY, np.argmax(preds_train, axis=1), labels=[0,1,2,3])

SyntaxError: ignored

In [ ]:
import pickle as pk

with open(preprocessed_data+"Bestmodel_golve_56 fscore.pkl", 'wb') as fb:
    pk.dump(model, fb)

In [ ]:
#@title different Bilstm for title, body and answer parts of a KU
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.metrics import classification_report
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional

# nb_words=len(word_index)+1

nb_words=MAX_NB_WORDS+1
embedding_dim=300
embedding_layer = Embedding(
    nb_words,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(word_embedding_matrix),
    trainable=False


)
neuron=128
shared_lstm1 = Bidirectional(LSTM(10, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
shared_lstm2 = Bidirectional(LSTM(60, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
shared_lstm3 = Bidirectional(LSTM(120, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))

# create model
sequence_1_Title = Input(shape=(MAX_SEQUENCE_LENGTH_Title,), dtype='float32')
embedding_Title_1 = embedding_layer(sequence_1_Title)
lstm_layer_Title_1=shared_lstm1(embedding_Title_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Title = Input(shape=(MAX_SEQUENCE_LENGTH_Title,), dtype='float32')
embedding_Title_2 = embedding_layer(sequence_2_Title)
lstm_layer_Title_2=shared_lstm1(embedding_Title_2)
#############
sequence_1_Body = Input(shape=(MAX_SEQUENCE_LENGTH_body,), dtype='float32')
embedding_Body_1 = embedding_layer(sequence_1_Body)
lstm_layer_Body_1=shared_lstm2(embedding_Body_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Body = Input(shape=(MAX_SEQUENCE_LENGTH_body,), dtype='float32')
embedding_Body_2 = embedding_layer(sequence_2_Body)
lstm_layer_Body_2=shared_lstm2(embedding_Body_2)



############
sequence_1_Answer = Input(shape=(MAX_SEQUENCE_LENGTH_AnswersBody,), dtype='float32')
embedding_Answer_1 = embedding_layer(sequence_1_Answer)
lstm_layer_Answer_1=shared_lstm3(embedding_Answer_1)
# dense=Dense(100, input_dim=embedding_dim, activation='relu')(embedding1)
sequence_2_Answer = Input(shape=(MAX_SEQUENCE_LENGTH_AnswersBody,), dtype='float32')
embedding_Answer_2 = embedding_layer(sequence_2_Answer)
lstm_layer_Answer_2=shared_lstm3(embedding_Answer_2)

#########


#########
# att1=keras.layers.MultiHeadAttention(num_heads=1,key_dim=1)(lstm_layer1,lstm_layer1,lstm_layer1)
# att2=keras.layers.MultiHeadAttention(num_heads=1,key_dim=1)(lstm_layer2,lstm_layer2,lstm_layer2)
flat1_t = keras.layers.Flatten()(lstm_layer_Title_1)
flat2_t = keras.layers.Flatten()(lstm_layer_Title_2)
flat1_b = keras.layers.Flatten()(lstm_layer_Body_1)
flat2_b = keras.layers.Flatten()(lstm_layer_Body_2)
flat1_a = keras.layers.Flatten()(lstm_layer_Answer_1)
flat2_a = keras.layers.Flatten()(lstm_layer_Answer_2)
t_cos=keras.layers.Dot(axes=1, normalize=True)([lstm_layer_Title_1,lstm_layer_Title_2])
flat_title_cosin = keras.layers.Flatten()(t_cos)
b_cos=keras.layers.Dot(axes=1, normalize=True)([lstm_layer_Body_1,lstm_layer_Body_2])
flat_body_cosin = keras.layers.Flatten()(b_cos)
a_cos=keras.layers.Dot(axes=1, normalize=True)([lstm_layer_Answer_1,lstm_layer_Answer_2])
flat_answer_cosin = keras.layers.Flatten()(a_cos)
# print("X",train_data_1[0])
# merged = keras.layers.concatenate([lstm_layer1, lstm_layer2], axis=-1)

merged = keras.layers.concatenate([flat_title_cosin,flat_body_cosin,flat1_t, flat_answer_cosin,flat2_t,flat1_b, flat2_b,flat1_a, flat2_a])
merged=Dense(50,activation='relu')(merged)

# merged = keras.layers.Flatten()(merged)
merged=keras.layers.Dropout(0.2)(merged)
output=Dense(4, activation='softmax')(merged)
model = Model(inputs=[sequence_1_Title,sequence_2_Title,sequence_1_Body,sequence_2_Body,sequence_1_Answer,sequence_2_Answer],outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history=model.fit([q1_train_Title, q2_train_Title, q1_train_body,q2_train_body,q1_train_AnswersBody,q2_train_AnswersBody]\
                  ,train_Y, validation_data=([q1_test_Title, q2_test_Title, q1_test_body,q2_test_body,q1_test_AnswersBody,q2_test_AnswersBody],test_Y),\
                  epochs=15, batch_size=512)#class_weight={0:.4,1:.2,2:.2,3:.3}
model.save_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')

Epoch 1/15


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report
model.load_weights('wikipedia-pubmed-and-PMC-w2v.bin.h5')

preds = model.predict([q1_test_Title, q2_test_Title, q1_test_body,q2_test_body,q1_test_AnswersBody,q2_test_AnswersBody],batch_size=512)
preds_train = model.predict([q1_train_Title, q2_train_Title, q1_train_body,q2_train_body,q1_train_AnswersBody,q2_train_AnswersBody],batch_size=512)

In [ ]:
preds

In [ ]:
print("For Test",classification_report(testY, np.argmax(preds, axis=1), labels=[0,1,2,3]))
print("For Train",classification_report(trainY, np.argmax(preds_train, axis=1), labels=[0,1,2,3]))

In [ ]:
# !wget https://nlp.stanford.edu/data/glove.840B.300d.zip


In [ ]:
# !wget https://zenodo.org/record/4641569/files/vectors.zip?download=1   ##Word Embedding trained on stack overflow

In [ ]:
# !unzip -q glove.840B.300d.zip ##vectors.zip?download=1

In [ ]:
# EMBEDDING_DIM = 300
# GLOVE_FILE='glove.840B.300d.txt'    #'glove.840B.300d.txt'
# print("Processing", GLOVE_FILE)

# embeddings_index = {}
# with open(GLOVE_FILE, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         embedding = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = embedding

# print('Word embeddings: %d' % len(embeddings_index))

In [ ]:
import csv
import pandas as pd

In [ ]:
# i=0
# keys=[]
# for key , value in embeddings_index.items():
#   if(i<10000):
#     i=i+1
#     keys.append(key)
#     # print (key)
# my_df = pd.DataFrame(keys)
# my_df.to_csv('my_csv.csv', index=False, header=False)
# # with open("output.csv", "wb") as f:
# #     writer = csv.writer(f)
# #     writer.writerows(keys)

In [ ]:
# embeddings_index["aspnet"]

GLOVE

In [ ]:
nb_words = min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    unknown_vector = embeddings_index.get("java")
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector
    else:
#         word_embedding_matrix[i] =unknown_vector
        print(word)

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

padding

In [ ]:

print(train_q1_Body[0])
q1_train_body[0]

how convert ftp get modification time file readable format hi need get last modified time files ftp server tried two ways none work cc first tried get modification time method like as result receive time cc cc cc cc correct date cc cc cc cc then tried mdtm method however one returns int in case returned cc any ideas get correct modified time convert readable format thank


array([    0,     0,     0,    11,   220,  1708,     6,  2107,    44,
           7,  2295,   200,  1143,    21,     6,   312,  1108,    44,
          86,  1708,    26,    42,    74,   868,   885,    46,     1,
          80,    42,     6,  2107,    44,    12,     9,   243,   115,
         583,    44,     1,     1,     1,     1,   219,   159,     1,
           1,     1,     1,   396,    42, 37536,    12,    97,    15,
         246,   224,    71,   141,   624,     1,   118,   469,     6,
         219,  1108,    44,   220,  2295,   200,   324], dtype=int32)

file names:


In [ ]:
weight ="glove_weight1060180_nb=14609_T+B_medium/"
weight="/drive/My Drive/Colab Notebooks/Pakrooh/"
q1_train_body_FILE =weight+"q1_train_body_FILE.npy"

q2_train_body_FILE =weight+"q2_train_body_FILE.npy"

q1_test_body_FILE =weight+"q1_test_body_FILE.npy"

q2_test_body_FILE =weight+"q2_test_body_FILE.npy"


train_labels_FILE =weight+"train_labels_FILE.npy"
test_labels_FILE =weight+"test_labels_FILE.npy"
# dev_labels_FILE =weight+"dev_labels_FILE.npy"

WORD_EMBEDDING_MATRIX_FILE = weight+"WORD_EMBEDDING_MATRIX_FILE.npy"
NB_WORDS_DATA_FILE = weight+'nb_words.json'

In [ ]:
print(test_labels[:2])

['duplicate\n', 'isolated\n']


In [ ]:
print(test_labels[:2])
print("test class ", len(test_labels))
print("train class ", len(train_labels))
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(train_labels[:40])
train_labels = lb.transform(train_labels)


lbT = preprocessing.LabelBinarizer()
lbT.fit(test_labels[:40])
test_labels = lb.transform(test_labels)


print('Shape of label tensor:', train_labels.shape)
print('Shape of label tensor:', test_labels.shape)

print(test_labels[:5])

['duplicate\n', 'isolated\n']
test class  8000
train class  32000
Shape of label tensor: (32000, 4)
Shape of label tensor: (8000, 4)
[[0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]]


Saving

In [ ]:
np.save(open(q1_train_body_FILE, 'wb'), q1_train_body)

np.save(open(q2_train_body_FILE, 'wb'), q2_train_body)

np.save(open(q1_test_body_FILE, 'wb'), q1_test_body)

np.save(open(q2_test_body_FILE, 'wb'), q2_test_body)

np.save(open(train_labels_FILE, 'wb'), train_labels)
np.save(open(test_labels_FILE, 'wb'), test_labels)


np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)

with open(NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)